# **ZincSight**: Interpretable prediction of zinc ion locations in proteins

⚠️ **Important Instructions:**

- If ZincSight crashes, please reset the runtime manually:
  Go to `Runtime` → `Disconnect and delete runtime`, then refresh the page and try again.
  <p></p>
- If the issue continues, contact **mechtinger1@mail.tau.ac.il** and include the **protein IDs or structures** you used for reproduction and debugging.
  <p></p>
- For maximum speed, change the hardware accelerator: Go to `Runtime` → `Change runtime type` → select `TPU`.
  <p></p>



In [1]:
#@title Query for ZincSight {display-mode: "form"}
#@markdown   **Enter your protein(s)** in the field below or upload **PDB** or **MMCIF** structures.
#@markdown - Query IDs can correspond to structures from **PDB** (can be either asymmetric units or biological assemblies), **AlphaFoldDB** (UniProt ID or AlphaFold model ID), or the **ESM Metagenomic Atlas**. The default field includes an example for each.
#@markdown - To upload your own structure files, check the appropriate box below and run this cell.
#@markdown ---
#@markdown <p></p>

import os
import shutil

query_dir_path = '/content/query_structures'
if os.path.exists(query_dir_path):
    print("Initialize query")
    for item_name in os.listdir(query_dir_path):
        item_path = os.path.join(query_dir_path, item_name)
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)
            # If it's a subdirectory, delete it and all its contents recursively
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)

structure_ids_for_download = '8QEP, 2A0S-assembly1, 1KLS, P0A6G5, AF-A0A068N621-F1-model_v6, MGYP002718891411' # @param {type:"string"}
upload_structures = False # @param {type:"boolean"}
if structure_ids_for_download.strip() == '' or upload_structures:
  os.makedirs(query_dir_path, exist_ok=True)
  %cd /content/query_structures
  from google.colab import files
  uploaded = files.upload()
  %cd /content
include_histidine_rotamers = True # @param {type: "boolean"}

In [ ]:
#@title Execute ZincSight {display-mode: "form"}
from IPython.utils.capture import capture_output
import os

SETUP_MARKER = "/content/ENV_SETUP.marker" # absolute marker so %cd won’t break it

# One-time environment setup (runs only first time):
if not os.path.exists(SETUP_MARKER):
    print(
        "🔧 First-time setup detected!\n"
        "Running this cell for the first time will take time due to dependencies installation.\n"
        "Please wait patiently while required packages and databases are configured."
    )
    with capture_output() as captured_output:

        # Clone the GitHub repository and change into the repo directory
        !git clone https://github.com/MECHTI1/ZincSight.git
        %cd ZincSight/

        #Setup environment
        !pip install -r requirements.txt

        # Run ZincSight
        import os
        print("Current Path:", os.getcwd()) # Debugging: Print current working directory

        import sys
        sys.path.append('/content/ZincSight')

    open(SETUP_MARKER, "w").close()

import multiprocessing, platform
def check_cpu():
    def count_physical_cores():
        phy = set()
        with open('/proc/cpuinfo') as f:
            phys_id = core_id = None
            for line in f:
                if line.startswith('physical id'):
                    phys_id = line.split(':',1)[1].strip()
                elif line.startswith('core id'):
                    core_id = line.split(':',1)[1].strip()
                if phys_id and core_id:
                    phy.add((phys_id, core_id))
                    phys_id = core_id = None
        return len(phy)

    logical = multiprocessing.cpu_count()
    physical = count_physical_cores()
    model = platform.processor() or "Unknown"
    arch, _ = platform.architecture()
    print(f"Logical cores (vCPUs): {logical}\nPhysical cores: {physical}\nModel: {model}\nArch: {arch}")

check_cpu()

# 1) cd into the cloned repo so all src/… paths line up:
%cd /content/ZincSight

# 2) ensure Python loads your local code first
import sys
sys.path.insert(0, "/content/ZincSight")

with capture_output() as captured_output:

    !touch ENV_SET_UP

    from main_execute import execute_zincsight

    # Define paths explicitly
    query_structures_path, output_path  = '/content/query_structures', '/content/output' # Path to input structures

    # Ensure paths exist
    if not os.path.exists(query_structures_path):
        os.makedirs(query_structures_path)
    if not os.path.exists(output_path):
        os.makedirs(output_path)  # Create output directory if it doesn't exist


    # Call the function with explicit paths
    output_compressed_file = execute_zincsight(include_histidine_rotamers, structure_ids_for_download, query_structures_path,output_path)

In [ ]:
#@title Download Results {display-mode: "form"}
if output_compressed_file!= False:
    from google.colab import files
    files.download(output_compressed_file)
else: print ("No predicted zinc-binding sites within the given query structures!")